In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import multirtd.params as params
from multirtd.dubins_planner import DubinsPlanner
from multirtd.dubins_model import Turtlebot

%load_ext autoreload
%autoreload 2

Test scenario with obstacles

In [ ]:
x0 = np.array([-5, 0, 0])

planner = DubinsPlanner()
# Set planner goal and obstacles
planner.p_goal = np.array([5, 0])
# (center, radius) tuples
obs_R = 0.5
planner.obstacles = [(np.array([3, 1]), obs_R),
                     (np.array([0, -1.5]), obs_R),
                     (np.array([-3, 1]), obs_R)]

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(x0[:-1]), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

plt.show()

In [ ]:
turtlebot = Turtlebot(x0=x0)
turtlebot.sigma = 0.01

controller_hz = 10
planner_hz = 2

t_sim = 0
planning_times = []

done = False
pbar = tqdm()
while not done:
    
    if t_sim % int(controller_hz/planner_hz) == 0:
        start_time = time.time()
        u = planner.traj_opt_sample(turtlebot.x, 0)
        planning_times.append(time.time() - start_time)

    turtlebot.step(u)
    #print(turtlebot.x)
    t_sim += 1

    if np.linalg.norm(turtlebot.x[:-1] - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True
    
    pbar.update(t_sim)

print("Avg planning time = ", np.mean(planning_times), " seconds")

In [ ]:
traj = np.stack(turtlebot.x_hist)[:,:-1]

# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(x0[:-1]), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r', alpha=0.5))

ax.plot(traj[:,0], traj[:,1])
for i in range(len(traj)):
    ax.add_patch(plt.Circle(tuple(traj[i]), params.R_BOT, color='b', alpha=0.02))

plt.show()

Test scenario with multiple obstacles and uncertainty regions

In [ ]:
planner.obstacles = [(np.array([0, 0]), 1.0)]

# Uncertainty region
unc_regions = [(np.array([0, 0]), 2.0)]

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(x0[:-1]), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

# Single obstacle with uncertainty region around it
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r', alpha=0.5))

for reg in unc_regions:
    ax.add_patch(plt.Circle(tuple(reg[0]), reg[1], color='r', alpha=0.1))

plt.show()

In [ ]:
turtlebot = Turtlebot(x0=x0)
turtlebot.sigma = 0.01

controller_hz = 10
planner_hz = 2

t_sim = 0
planning_times = []

done = False
pbar = tqdm()
while not done:
    
    if t_sim % int(controller_hz/planner_hz) == 0:
        start_time = time.time()
        u = planner.traj_opt_sample(turtlebot.x, 0)
        planning_times.append(time.time() - start_time)

    turtlebot.step(u)
    #print(turtlebot.x)
    t_sim += 1

    if np.linalg.norm(turtlebot.x[:-1] - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True
    
    pbar.update(t_sim)

print("Avg planning time = ", np.mean(planning_times), " seconds")